In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVR
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import math
from sklearn.preprocessing import StandardScaler
import random
import sklearn
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [2]:
# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
Nasa=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\cocomonasa_2_dataset.csv",header=None,index_col=0)
columns_Nasa=['ProjName','CatofApp','ForG','Center','YearofDev','DevMode','rely','data','cplx','time','stor','virt','turn','acap','aexp','pcap','vexp','lexp','modp','tool','sced','loc','Effort']
Nasa.set_axis(columns_Nasa,axis='columns',inplace=True)
Nasa.rename_axis("Features", axis=1,inplace=True)
Nasa.rename_axis("Projects ID", axis=0,inplace=True)
Nasa.reset_index(inplace=True)
Nasa.drop(columns=['Projects ID'], inplace=True)

#Nasa dataset preprocessing
def NASA_quantifier(x):
    if x=='vl':
        return 0
    elif x=='l':
        return 1
    elif x=='n':
        return 2
    elif x=='h':
        return 3
    elif x=='vh':
        return 4
    elif x=='xh':
        return 5
    else:
        return x

Nasa=Nasa.applymap(NASA_quantifier)
Nasa=pd.get_dummies(Nasa)
dataset = Nasa

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort'])  # Features
y = dataset['Effort']  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.function(reduce_retracing=True)
# Split data into train and test sets

# Create a classifier
estimator = SVR(kernel='linear')

# Create SFS model
sfs = SequentialFeatureSelector(estimator,n_features_to_select='auto', direction='forward', cv=5 , scoring='neg_mean_absolute_error')

# Fit SFS model on training data
sfs.fit(X, y)

# Transform the data to selected features
X_selected = sfs.transform(X)
print(X_selected)

#=====================================================================================================================

# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
    'neurons_input': (10, 50),
    'neurons_hidden': (10, 50),
    'num_hidden_layers': (1, 5),
    'learning_rate': (1e-5, 1e-2),
    'batch_size': (8, 32),
    'epochs': (5, 20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)

    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )

    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
        
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)

    # Calculate the Root Mean Squared Error (RMSE) for this fold.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)

     # Convert y_pred and y_test to NumPy arrays for MMRE calculation
    y_pred = np.array(y_pred).flatten()
    y_test = np.array(y_test)
    
    # Calculate the Mean Magnitude of Relative Error (MMRE) for this fold.
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)
    
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")   
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

[[3. 2. 1. ... 0. 0. 1.]
 [3. 2. 1. ... 0. 0. 1.]
 [3. 2. 1. ... 0. 0. 1.]
 ...
 [3. 4. 3. ... 0. 1. 0.]
 [3. 4. 3. ... 0. 1. 0.]
 [3. 4. 3. ... 0. 1. 0.]]
|   iter    |  target   | batch_... |  epochs   | learni... | neuron... | neuron... | num_hi... |
-------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 69ms/step
| 1         | -435.1    | 16.99     | 19.26     | 0.007323  | 33.95     | 16.24     | 1.624     |
1/1 [==============================] - 0s 46ms/step
| 2         | -393.6    | 9.394     | 17.99     | 0.006015  | 38.32     | 10.82     | 4.88      |
1/1 [==============================] - 0s 43ms/step
| 3         | -460.3    | 27.98     | 8.185     | 0.001826  | 17.34     | 22.17     | 3.099     |
1/1 [==============================] - 0s 39ms/step
| 4         | -433.4    | 18.37     | 9.368     | 0.006122  | 15.58     | 21.69     | 2.465     |
1/1 [==============================] - 0s 35ms/s

1/1 [==============================] - 0s 36ms/step
| 28        | -546.5    | 10.51     | 16.13     | 0.005838  | 46.93     | 17.55     | 1.793     |
1/1 [==============================] - 0s 33ms/step
| 29        | -715.1    | 29.95     | 6.991     | 0.004999  | 41.0      | 30.14     | 1.329     |
1/1 [==============================] - 0s 45ms/step
| 30        | -716.1    | 23.13     | 8.055     | 0.003133  | 17.94     | 16.38     | 4.888     |
1/1 [==============================] - 0s 32ms/step
| 31        | -717.1    | 27.88     | 15.54     | 0.001176  | 26.81     | 12.74     | 1.15      |
1/1 [==============================] - 0s 43ms/step
| 32        | -594.8    | 14.85     | 13.56     | 0.007422  | 46.53     | 13.17     | 3.828     |
1/1 [==============================] - 0s 39ms/step
| 33        | -546.9    | 20.87     | 19.84     | 0.006639  | 10.14     | 30.0      | 2.181     |
1/1 [==============================] - 0s 44ms/step
| 34        | -504.0    | 9.572     | 9.772     

1/1 [==============================] - 0s 37ms/step
| 25        | -2.026e+0 | 20.57     | 7.095     | 0.004777  | 31.34     | 14.44     | 2.192     |
1/1 [==============================] - 0s 44ms/step
| 26        | -2.038e+0 | 13.42     | 6.584     | 0.001587  | 31.42     | 41.7      | 4.011     |
1/1 [==============================] - 0s 37ms/step
| 27        | -1.501e+0 | 13.33     | 14.39     | 0.005921  | 10.51     | 43.01     | 2.079     |
1/1 [==============================] - 0s 33ms/step
| 28        | -1.52e+03 | 10.51     | 16.13     | 0.005838  | 46.93     | 17.55     | 1.793     |
1/1 [==============================] - 0s 33ms/step
| 29        | -2.038e+0 | 29.95     | 6.991     | 0.004999  | 41.0      | 30.14     | 1.329     |
1/1 [==============================] - 0s 42ms/step
| 30        | -2.035e+0 | 23.13     | 8.055     | 0.003133  | 17.94     | 16.38     | 4.888     |
1/1 [==============================] - 0s 32ms/step
| 31        | -2.042e+0 | 27.88     | 15.54     

1/1 [==============================] - 0s 37ms/step
| 22        | -1.165e+0 | 10.85     | 10.28     | 0.007454  | 37.26     | 32.57     | 2.429     |
1/1 [==============================] - 0s 33ms/step
| 23        | -1.705e+0 | 16.34     | 15.16     | 0.004878  | 13.67     | 10.61     | 1.883     |
1/1 [==============================] - 0s 39ms/step
| 24        | -1.317e+0 | 23.59     | 18.05     | 0.005597  | 23.23     | 48.02     | 3.229     |
1/1 [==============================] - 0s 36ms/step
| 25        | -1.708e+0 | 20.57     | 7.095     | 0.004777  | 31.34     | 14.44     | 2.192     |
1/1 [==============================] - 0s 43ms/step
| 26        | -1.719e+0 | 13.42     | 6.584     | 0.001587  | 31.42     | 41.7      | 4.011     |
1/1 [==============================] - 0s 38ms/step
| 27        | -1.417e+0 | 13.33     | 14.39     | 0.005921  | 10.51     | 43.01     | 2.079     |
1/1 [==============================] - 0s 32ms/step
| 28        | -1.349e+0 | 10.51     | 16.13     

1/1 [==============================] - 0s 49ms/step
| 19        | -522.7    | 20.06     | 7.734     | 0.003876  | 41.29     | 46.86     | 4.87      |
1/1 [==============================] - 0s 40ms/step
| 20        | -485.1    | 16.78     | 19.67     | 0.00604   | 32.85     | 16.21     | 2.199     |
1/1 [==============================] - 0s 35ms/step
| 21        | -401.8    | 16.62     | 18.92     | 0.01      | 34.74     | 16.39     | 1.0       |
1/1 [==============================] - 0s 45ms/step
| 22        | -307.8    | 20.01     | 6.956     | 0.00648   | 40.08     | 48.18     | 3.636     |
1/1 [==============================] - 0s 40ms/step
| 23        | -601.9    | 19.02     | 6.464     | 0.006308  | 40.24     | 49.07     | 2.981     |
1/1 [==============================] - 0s 35ms/step
| 24        | -447.9    | 17.42     | 19.43     | 0.01      | 33.89     | 17.15     | 1.0       |
1/1 [==============================] - 0s 34ms/step
| 25        | -502.6    | 16.99     | 19.15     

In [3]:
print(sfs.get_feature_names_out())

['rely' 'time' 'virt' 'vexp' 'sced' 'loc' 'ProjName_Y' 'ProjName_gal'
 'ProjName_hst' 'ProjName_spl' 'CatofApp_Avionics'
 'CatofApp_application_ground' 'CatofApp_datacapture'
 'CatofApp_missionplanning' 'CatofApp_monitor_control'
 'CatofApp_operatingsystem' 'CatofApp_realdataprocessing'
 'CatofApp_science' 'CatofApp_simulation' 'CatofApp_utility' 'ForG_f'
 'ForG_g']


In [2]:
# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
Nasa=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\cocomonasa_2_dataset.csv",header=None,index_col=0)
columns_Nasa=['ProjName','CatofApp','ForG','Center','YearofDev','DevMode','rely','data','cplx','time','stor','virt','turn','acap','aexp','pcap','vexp','lexp','modp','tool','sced','loc','Effort']
Nasa.set_axis(columns_Nasa,axis='columns',inplace=True)
Nasa.rename_axis("Features", axis=1,inplace=True)
Nasa.rename_axis("Projects ID", axis=0,inplace=True)
Nasa.reset_index(inplace=True)
Nasa.drop(columns=['Projects ID'], inplace=True)

#Nasa dataset preprocessing
def NASA_quantifier(x):
    if x=='vl':
        return 0
    elif x=='l':
        return 1
    elif x=='n':
        return 2
    elif x=='h':
        return 3
    elif x=='vh':
        return 4
    elif x=='xh':
        return 5
    else:
        return x

Nasa=Nasa.applymap(NASA_quantifier)
Nasa=pd.get_dummies(Nasa)
dataset = Nasa

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort'])  # Features
y = dataset['Effort']  # Target (effort)

In [4]:
X

,Center,YearofDev,rely,data,cplx,time,stor,virt,turn,acap,...,CatofApp_operatingsystem,CatofApp_realdataprocessing,CatofApp_science,CatofApp_simulation,CatofApp_utility,ForG_f,ForG_g,DevMode_embedded,DevMode_organic,DevMode_semidetached
0,2,1979,3,1,3,2,2,1,1,2,...,0,0,0,0,0,0,1,0,0,1
1,2,1979,3,1,3,2,2,1,1,2,...,0,0,0,0,0,0,1,0,0,1
2,2,1979,3,1,3,2,2,1,1,2,...,0,0,0,0,0,0,1,0,0,1
3,2,1979,3,1,3,2,2,1,1,2,...,0,0,0,0,0,0,1,0,0,1
4,2,1979,3,1,3,2,2,1,1,2,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,5,1982,4,1,4,4,5,1,1,3,...,0,0,0,0,0,1,0,1,0,0
89,5,1980,4,3,4,5,5,2,2,3,...,0,0,0,0,0,1,0,1,0,0
90,2,1983,3,2,4,4,4,3,3,2,...,0,0,0,0,0,1,0,1,0,0
91,2,1983,3,2,4,4,4,3,3,2,...,0,0,0,0,0,1,0,1,0,0
